# week 03 part 3

## 1. setting up environment

In [1]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)

import json

!pip install geopandas
!pip install geopy
!pip install beautifulsoup4
!pip install html5lib
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

print("Library imported.")

Library imported.


## 2. Scrape thWikipedia page

In [2]:
 data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
read_table = pd.read_html(url,header=[0])
df = read_table[0]
df.head(10)

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
7,M8A,Not assigned,NaN
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge


## 3. modifying the dataframe
The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [4]:
#rename columns
df = df.rename(index=str, columns={'Postcode':'PostalCode','Neighbourhood':'Neighborhood'})
df.head()



,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


Only process the cells that have an assigned borough. Ignore cells with a borough that is *Not assigned*

In [5]:
#drop not assigned
df = df[df.Borough !='Not assigned']
df.reset_index(drop=True,inplace=True)
df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


More than one neighborhood can exist in one postal code area, combine rows

In [6]:
#merge by postal code  
df = df.groupby('Postal code',as_index=False).agg(lambda x: ','.join(set(x.dropna())))

df.head()

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Borough not assigned neighborhood

In [7]:
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
df.loc[df['Neighborhood'] == 'Not assigned','Neighborhood'] = df['Borough']
#rename to make column name correct
df.rename(columns={"Postal code": "PostalCode"}, inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Show the number of rows

In [8]:
df.shape[0]

103

## 3.  create df containing geographical coordinates

In [9]:
# load the coordinates from the csv file on Coursera
df_geo = pd.read_csv("Geospatial_Coordinates.csv")
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
#initiating l and l
df['Latitude'] = '0';
df['Longitude'] = '0';

#adding coordinates
for i in df.index:
    for j in df_geo.index:
        if df.iloc[i, 0] == df_geo.iloc[j, 0]:
            df.iloc[i, 3] = df_geo.iloc[j, 1]
            df.iloc[i, 4] = df_geo.iloc[j, 2]
            
df.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.8067,-79.1944
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.7845,-79.1605
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395


## 4. cluster the neighborhood whose Borough containing "Toronto"

In [11]:
dfa = df.copy()
dfa= dfa[df.Borough.str.contains("Toronto")]
dfa.tail()



,PostalCode,Borough,Neighborhood,Latitude,Longitude
82,M6P,West Toronto,High Park / The Junction South,43.6616,-79.4648
83,M6R,West Toronto,Parkdale / Roncesvalles,43.649,-79.4563
84,M6S,West Toronto,Runnymede / Swansea,43.6516,-79.4844
85,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.6623,-79.3895
87,M7Y,East Toronto,Business reply mail Processing CentrE,43.6627,-79.3216


In [12]:
#set up and modify map
toronto_map = folium.Map(location=[43.65, -79.4], zoom_start=12)

X = dfa['Latitude']
Y = dfa['Longitude']
Z = np.stack((X, Y), axis=1)

kmeans = KMeans(n_clusters=5, random_state=0).fit(Z)

clusters = kmeans.labels_
colors = ['red', 'green', 'blue', 'yellow']
dfa['Cluster'] = clusters

for latitude, longitude, borough, cluster in zip(dfa['Latitude'], dfa['Longitude'], dfa['Borough'], dfa['Cluster']):
    label = folium.Popup(borough, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  

toronto_map

## 5. find top 50 venues within a radius of 5000m w.r.t. Queen's Park / Ontario Provincial Government

In [13]:
lat = dfa.loc[85, 'Latitude'] # neighborhood latitude value, find Queen's location
long = dfa.loc[85, 'Longitude'] # neighborhood longitude value

neigh = dfa.loc[85, 'Neighborhood'] # neighborhood name

print('{} has a latitude and longitude value of {} and {}.'.format(neigh, 
                                                               lat, 
                                                              long))

Queen's Park / Ontario Provincial Government has a latitude and longitude value of 43.6623015 and -79.3894938.


In [14]:
#define foursquare credentials

CLIENT_ID = 'ADEAGQE5LK3IKPH33ONIL5DTABARUS04RI2W0EAUE3JV3QRH' 
CLIENT_SECRET = 'W5TPIHHRPL3XIGYRKGZQ1DQAG1IDPXJ3KW45ZEONANUSZBP2' 
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)




Your credentails:
CLIENT_ID: ADEAGQE5LK3IKPH33ONIL5DTABARUS04RI2W0EAUE3JV3QRH
CLIENT_SECRET:W5TPIHHRPL3XIGYRKGZQ1DQAG1IDPXJ3KW45ZEONANUSZBP2


In [15]:
#define url of foursquare
LIMIT = 50
radius = 5000

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    long, 
    radius, 
    LIMIT)
url # display URL



'https://api.foursquare.com/v2/venues/explore?&client_id=ADEAGQE5LK3IKPH33ONIL5DTABARUS04RI2W0EAUE3JV3QRH&client_secret=W5TPIHHRPL3XIGYRKGZQ1DQAG1IDPXJ3KW45ZEONANUSZBP2&v=20180605&ll=43.6623015,-79.3894938&radius=5000&limit=50'

In [16]:
#get result
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ea978f6882fc706cbd5cb90'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 239,
  'suggestedBounds': {'ne': {'lat': 43.70730154500004,
    'lng': -79.32740548666185},
   'sw': {'lat': 43.617301454999954, 'lng': -79.45158211333815}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5a81ae339deb7d369fa7f146',
       'name': 'Hailed Coffee',
       'location': {'address': '44 Gerrard St W',
        'crossStreet': 'Yonge St',
        'lat': 43.65883296982352,
        'lng': -79.38368351986598,
        'labeledLatLngs': [{'label': 'display',
        

In [17]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:
venues = results['response']['groups'][0]['items']

from pandas.io.json import json_normalize

nearby_venues = pd.io.json.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name','venue.neighborhood','venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/Users/dongningsun/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,name,neighborhood,categories,lat,lng
0,Hailed Coffee,NaN,Coffee Shop,43.658833,-79.383684
1,DanceLifeX Centre,NaN,Dance Studio,43.666956,-79.385297
2,Como En Casa,NaN,Mexican Restaurant,43.665160,-79.384796
3,Queen's Park,NaN,Park,43.663946,-79.392180
4,Neo Coffee Bar,NaN,Coffee Shop,43.660140,-79.385870


In [19]:
#provide how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(nearby_venues['categories'].unique())))

There are 38 uniques categories.


## 6.Explore Neighborhood in Toronto

In [22]:
#define function to retrieve venues information for neighborhood in Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
#run the function to provide top 50 venues around each neighborhood in Toronto within radius of 5000m


trt_vn = getNearbyVenues(names=dfa['Neighborhood'],
                                   latitudes=dfa['Latitude'],
                                   longitudes=dfa['Longitude']
                                  )



The Beaches
The Danforth West / Riverdale
India Bazaar / The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park / Summerhill East
Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park
Rosedale
St. James Town / Cabbagetown
Church and Wellesley
Regent Park / Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond / Adelaide / King
Harbourfront East / Union Station / Toronto Islands
Toronto Dominion Centre / Design Exchange
Commerce Court / Victoria Hotel
Roselawn
Forest Hill North & West
The Annex / North Midtown / Yorkville
University of Toronto / Harbord
Kensington Market / Chinatown / Grange Park
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport
Stn A PO Boxes
First Canadian Place / Underground city
Christie
Dufferin / Dovercourt Village
Little Portugal / Trinity
Brockton / Parkdale Village / Exhibition Place
High Park / 

In [25]:
#provide the size of the returned venues
print(trt_vn.shape)
trt_vn.head()

(1212, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Danforth West / Riverdale,43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


In [26]:
#provide how many venues per each neighborhood
trt_vn.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,50,50,50,50,50,50
Brockton / Parkdale Village / Exhibition Place,23,23,23,23,23,23
Business reply mail Processing CentrE,16,16,16,16,16,16
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport,18,18,18,18,18,18
Central Bay Street,50,50,50,50,50,50


In [27]:
#print the amount of unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(trt_vn['Venue Category'].unique())))

There are 218 uniques categories.


## 7. Analyze Each Neighborhood

In [28]:
# one hot encoding
trt_onehot = pd.get_dummies(trt_vn[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
trt_onehot['Neighborhood'] = trt_vn['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [trt_onehot.columns[-1]] + list(trt_onehot.columns[:-1])
trt_onehot = trt_onehot[fixed_columns]

trt_grp = trt_onehot.groupby('Neighborhood').mean().reset_index()
trt_grp.head();
trt_grp.shape


manhattan_onehot.head()

(39, 218)

In [29]:
#print the top 5 venues around each neighbothood with their recurrence frequency 
num_top_venues = 5

for hood in trt_grp['Neighborhood']:
    print("----"+hood+"----")
    temp = trt_grp[trt_grp['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.08
1        Cocktail Bar  0.06
2  Seafood Restaurant  0.04
3              Bakery  0.04
4      Farmers Market  0.04


----Brockton / Parkdale Village / Exhibition Place----
            venue  freq
0            Café  0.13
1     Coffee Shop  0.09
2       Nightclub  0.09
3  Breakfast Spot  0.09
4             Gym  0.04


----Business reply mail Processing CentrE----
                venue  freq
0  Light Rail Station  0.12
1         Pizza Place  0.06
2              Garden  0.06
3          Restaurant  0.06
4       Burrito Place  0.06


----CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport----
              venue  freq
0   Airport Service  0.17
1    Airport Lounge  0.11
2  Airport Terminal  0.11
3   Harbor / Marina  0.06
4               Bar  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.22
1  Italian Restaurant  0.06
2  

In [30]:
#put information into dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [31]:
#get top 10 venues around each neighbothood with their recurrence frequency
# presented as a dataframe
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = trt_grp['Neighborhood']

for ind in np.arange(trt_grp.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(trt_grp.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Beer Bar,Seafood Restaurant,Restaurant,Italian Restaurant,Farmers Market,Café,Bakery
1,Brockton / Parkdale Village / Exhibition Place,Café,Nightclub,Coffee Shop,Breakfast Spot,Convenience Store,Climbing Gym,Stadium,Bakery,Intersection,Italian Restaurant
2,Business reply mail Processing CentrE,Light Rail Station,Park,Pizza Place,Fast Food Restaurant,Auto Workshop,Farmers Market,Spa,Restaurant,Smoke Shop,Comic Shop
3,CN Tower / King and Spadina / Railway Lands / ...,Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Sculpture Garden,Plane,Rental Car Location,Harbor / Marina,Boat or Ferry,Bar
4,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Bubble Tea Shop,Ice Cream Shop,Burger Joint,Diner,Bar,Hotel


## 8. cluster neighborhood

In [32]:
# set number of clusters
kclusters = 3

trt_grp_clustering = trt_grp.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(trt_grp_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [34]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

trt_merged = trt_vn

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
trt_merged = trt_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

trt_merged.head() 

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail,1,Pub,Trail,Health Food Store,Cuban Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store,1,Pub,Trail,Health Food Store,Cuban Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub,1,Pub,Trail,Health Food Store,Cuban Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood,1,Pub,Trail,Health Food Store,Cuban Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
4,The Danforth West / Riverdale,43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Bookstore,Indian Restaurant,Juice Bar,Brewery,Bubble Tea Shop


In [37]:

# create map
map_clusters = folium.Map(location=[43.65, -79.4], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
centroid_colors = ['red','green''yellow']

rainbow = [next(iter(centroid_colors),i) for i in range(kclusters)]
#it is very interesting that colors.rgb2hex is not foundable.


# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(trt_merged['Venue Latitude'], trt_merged['Venue Longitude'], trt_merged['Neighborhood'], trt_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster Labels' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 9. Examine clusters

In [38]:
trt_merged.loc[trt_merged['Cluster Labels'] == 0, trt_merged.columns[[1] + list(range(5, trt_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
106,43.728020,-79.394382,Park,0,Bus Line,Swim School,Park,College Gym,College Cafeteria,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
107,43.728020,-79.382860,Swim School,0,Bus Line,Swim School,Park,College Gym,College Cafeteria,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
108,43.728020,-79.382805,Bus Line,0,Bus Line,Swim School,Park,College Gym,College Cafeteria,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
175,43.689574,-79.383438,Park,0,Park,Playground,Tennis Court,Restaurant,Creperie,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop
176,43.689574,-79.384504,Restaurant,0,Park,Playground,Tennis Court,Restaurant,Creperie,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop
177,43.689574,-79.382900,Tennis Court,0,Park,Playground,Tennis Court,Restaurant,Creperie,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop
178,43.689574,-79.383465,Playground,0,Park,Playground,Tennis Court,Restaurant,Creperie,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop
195,43.679563,-79.378934,Playground,0,Park,Trail,Playground,Cuban Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
196,43.679563,-79.373788,Park,0,Park,Trail,Playground,Cuban Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
197,43.679563,-79.382773,Park,0,Park,Trail,Playground,Cuban Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


In [39]:
trt_merged.loc[trt_merged['Cluster Labels'] == 1, trt_merged.columns[[1] + list(range(5, trt_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,43.676357,-79.293942,Trail,1,Pub,Trail,Health Food Store,Cuban Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
1,43.676357,-79.297734,Health Food Store,1,Pub,Trail,Health Food Store,Cuban Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
2,43.676357,-79.297215,Pub,1,Pub,Trail,Health Food Store,Cuban Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
3,43.676357,-79.292869,Neighborhood,1,Pub,Trail,Health Food Store,Cuban Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
4,43.679557,-79.351265,Cosmetics Shop,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Bookstore,Indian Restaurant,Juice Bar,Brewery,Bubble Tea Shop
5,43.679557,-79.351434,Greek Restaurant,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Bookstore,Indian Restaurant,Juice Bar,Brewery,Bubble Tea Shop
6,43.679557,-79.350115,Italian Restaurant,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Bookstore,Indian Restaurant,Juice Bar,Brewery,Bubble Tea Shop
7,43.679557,-79.351187,Ice Cream Shop,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Bookstore,Indian Restaurant,Juice Bar,Brewery,Bubble Tea Shop
8,43.679557,-79.352295,Ice Cream Shop,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Bookstore,Indian Restaurant,Juice Bar,Brewery,Bubble Tea Shop
9,43.679557,-79.352116,Yoga Studio,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Bookstore,Indian Restaurant,Juice Bar,Brewery,Bubble Tea Shop


In [40]:
trt_merged.loc[trt_merged['Cluster Labels'] == 2, trt_merged.columns[[1] + list(range(5, trt_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
740,43.711695,-79.411978,Garden,2,Garden,Wine Shop,Cuban Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop
